In [28]:
from google.colab import files
uploaded = files.upload()

Saving datajoined.csv to datajoined (3).csv


In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import scipy
import seaborn as sns
import math
from scipy import stats

In [34]:
df = pd.read_csv('datajoined.csv')
df.head()

,Latitude,Longitude,Sum_INCENDIE_count,Sum_Infractions_entrainant_count,Sum_ALARMES_INCENDIES_count,Sum_total_crimes,Population,Dwellings,Households,Average age,median income,sum_etage_hors_sol,sum_nombre_logement,avg_annee_construction,sum_superficie_terrain,sum_superficie_batiment
0,45.40,-73.94,3,0,107,23,NaN,NaN,NaN,NaN,NaN,37.0,45.0,1929.333333,27416.0,4254.0
1,45.40,-73.95,2,0,38,91,NaN,NaN,NaN,NaN,NaN,426.0,574.0,1924.381148,124893.0,93948.0
2,45.40,-73.96,0,0,1,1,352.0,162.0,149.0,42.0,32032.0,0.0,0.0,1900.000000,2732.0,831.0
3,45.41,-73.88,0,0,0,1,NaN,NaN,NaN,NaN,NaN,11.0,6.0,1914.333333,17569.0,2112.0
4,45.41,-73.89,0,0,3,6,NaN,NaN,NaN,NaN,NaN,37.0,20.0,1961.100000,40154.0,6022.0


In [35]:
# Remove the rows with NaN
df = df.dropna()

In [ ]:
df['Sum_INCENDIE_count'].value_counts()

In [79]:
#Define three classes of fire risk by the summed number of fire incidence from 2015-2020
#low risk < 5, moderate risk 5-15, high risk > 15
df['fire'] = df['Sum_INCENDIE_count']
df['fire'][df['Sum_INCENDIE_count']<5] = 0
df['fire'][(df['Sum_INCENDIE_count']>=5) & (df['Sum_INCENDIE_count']<15)] = 1
df['fire'][df['Sum_INCENDIE_count']>=15] = 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [80]:
df['fire'].value_counts()

0    181
1    173
2    159
Name: fire, dtype: int64

In [81]:
# Standardize the features
from sklearn import preprocessing
raw_X = df[['Sum_Infractions_entrainant_count','Sum_ALARMES_INCENDIES_count','Sum_total_crimes','Population','Dwellings','Households','Average age','median income','sum_etage_hors_sol','sum_nombre_logement','avg_annee_construction','sum_superficie_terrain','sum_superficie_batiment']]
# standardize X and y before fitting multiple linear model
X = pd.DataFrame(preprocessing.scale(raw_X))
y = df['fire']

In [168]:
#Leave out 10% data for validation
X_valid=X.sample(frac=0.1,random_state=200)
y_valid=y.sample(frac=0.1,random_state=200)
#All other data used for building model
X_whole=X.drop(X_valid.index)
y_whole=y.drop(y_valid.index)

In [169]:
X_whole.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.395822,-0.750164,-0.804202,-0.835903,-0.709150,-0.779266,0.083633,-0.336881,-1.193408,-0.971973,-4.034922,-1.800365,-0.263633
1,-0.395822,-0.556602,-0.706099,-0.393300,-0.635599,-0.665816,1.590483,0.208687,-0.886426,-0.847526,-0.367542,-0.087765,-0.240826
2,-0.395822,-0.685643,-0.703127,-0.270578,-0.353655,-0.344376,0.547279,0.319256,-0.856690,-0.824399,-0.708705,0.044010,-0.240909
3,-0.395822,-0.658760,-0.801229,0.735338,0.046788,0.099969,-0.694630,1.608201,-1.188160,-0.970321,-3.154571,-1.680173,-0.259594
4,-0.395822,-0.056566,-0.379092,-0.288684,-0.353655,-0.316013,0.530720,0.178876,0.335378,0.320405,0.318420,-0.032370,-0.147684


In [179]:
# Evaluate models using 5-fold cross-validation
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# create dataset
X_whole, y_whole = make_classification(random_state=0)
# prepare the cross-validation procedure
cv = KFold(n_splits=5, random_state=1, shuffle=True)

Approach #1: logistic regression

In [180]:
from sklearn.linear_model import LogisticRegression
# create model
logreg = LogisticRegression()
# evaluate model
logreg_scores = cross_val_score(logreg, X_whole, y_whole, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(logreg_scores), std(logreg_scores)))

Accuracy: 0.820 (0.051)


Approach #2: XGboost classifier

In [181]:
from xgboost import XGBClassifier
# create model
xgboost = XGBClassifier()
# evaluate model
xgboost_scores = cross_val_score(xgboost, X_whole, y_whole, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(xgboost_scores), std(xgboost_scores)))

Accuracy: 0.910 (0.049)


The accuary of XGboost classifier is higher than logistic regression classifier so we will use XGboost for validation sets.

In [171]:
xgboost_final = XGBClassifier()
xgboost_final.fit(X_whole, y_whole)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [172]:
y_valid_pred = xgboost_final.predict(X_valid)

In [174]:
print('Accuracy of XGboost classifier on test set: {:.2f}'.format(xgboost_final.score(X_valid, y_valid)))

Accuracy of XGboost classifier on test set: 0.80


In [175]:
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print(confusion_matrix_valid)

[[11  3  0]
 [ 2 10  3]
 [ 0  2 20]]
